frame yakalam

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    cv2.imshow("Webcam",frame)
    
    a = cv2.waitKey(0)
    if a == ord('q'):
        break
    elif a == ord('s'):
        cv2.imwrite("webcam.png",frame)
        
cap.release()
cv2.destroyAllWindows()


hsv ile renk takibi

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

def nothing(x):
    pass

cv2.namedWindow("Trackbar")
cv2.resizeWindow("Trackbar", 400,400)

cv2.createTrackbar("Lower - H", "Trackbar", 0, 180, nothing)
cv2.createTrackbar("Lower - S", "Trackbar", 0, 255, nothing)
cv2.createTrackbar("Lower - V", "Trackbar", 0, 255, nothing)

cv2.createTrackbar("Upper - H", "Trackbar", 0, 180, nothing)
cv2.createTrackbar("Upper - S", "Trackbar", 0, 255, nothing)
cv2.createTrackbar("Upper - V", "Trackbar", 0, 255, nothing)

cv2.setTrackbarPos("Upper - H", "Trackbar", 180)
cv2.setTrackbarPos("Upper - S", "Trackbar", 255)
cv2.setTrackbarPos("Upper - V", "Trackbar", 255)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_h = cv2.getTrackbarPos("Lower - H", "Trackbar")
    lower_s = cv2.getTrackbarPos("Lower - S", "Trackbar")
    lower_v = cv2.getTrackbarPos("Lower - V", "Trackbar")

    upper_h = cv2.getTrackbarPos("Upper - H", "Trackbar")
    upper_s = cv2.getTrackbarPos("Upper - S", "Trackbar")
    upper_v = cv2.getTrackbarPos("Upper - V", "Trackbar")

    lower_color = np.array([lower_h, lower_s, lower_v])
    upper_color = np.array([upper_h, upper_s, upper_v])

    mask = cv2.inRange(frame_hsv, lower_color, upper_color)

    cv2.imshow("Original", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(20) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

para sayma1

In [1]:
import cv2
import numpy as np

img1 = cv2.imread("para1.jpg")
img1 = cv2.resize(img1,(640,480))
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img1_blur = cv2.medianBlur(gray1,19)

circles1 = cv2.HoughCircles(img1_blur,cv2.HOUGH_GRADIENT,1,img1.shape[0]/20,param1=180,param2=20,minRadius=20,maxRadius=97)

if circles1 is not None:
    circles1 = np.uint16(np.around(circles1))
    
    for i in circles1[0,:]: 
        cv2.circle(img1,(i[0],i[1]),i[2],(0,255,0),2)
        coins = len(circles1[0]) 
    
    cv2.putText(img1, f"Bulunan para sayisi: {coins}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)        

cv2.imshow("img1",img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

para sayma2

In [2]:
import cv2
import numpy as np

img2 = cv2.imread("para2.jpg")
img2 = cv2.resize(img2, (640, 480))
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2_blur = cv2.medianBlur(gray2, 19)

circles2 = cv2.HoughCircles(img2_blur, cv2.HOUGH_GRADIENT, 1, img2.shape[0]/5, param1=200, param2=20, minRadius=50, maxRadius=117)

if circles2 is not None:
    circles2 = np.uint16(np.around(circles2))
    coins = len(circles2[0])
    
    for i in circles2[0, :]:
        cv2.circle(img2, (i[0], i[1]), i[2], (0, 255, 0), 2)
    
    cv2.putText(img2, f"Bulunan para sayisi: {coins}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

cv2.imshow("img2", img2)

cv2.waitKey(0)
cv2.destroyAllWindows()


para sayma3


In [ ]:
import cv2
import numpy as np

img3= cv2.imread("para3.jpg")
img3 = cv2.resize(img3,(640,480))
gray3 = cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
img3_blur = cv2.medianBlur(gray1,5)

circles3 = cv2.HoughCircles(img3_blur,cv2.HOUGH_GRADIENT,1,img3.shape[0]/8,param1=180,param2=20,minRadius=20,maxRadius=110)

if circles3 is not None:
    circles3 = np.uint16(np.around(circles3))
    
    for i in circles1[0,:]: 
        cv2.circle(img3,(i[0],i[1]),i[2],(0,255,0),2) 
        coins = len(circles3[0])
    
    cv2.putText(img3, f"Bulunan para sayisi: {coins}", (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        
cv2.imshow("img3",img3)

cv2.waitKey(0)
cv2.destroyAllWindows()

para Watershed1

In [1]:
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage
import numpy as np
import imutils
import cv2

image = cv2.imread("para3.jpg")

# Ortalama Kaydırma (Mean Shift) filtreleme uygula / görüntüdeki renk varyasyonlarını azaltarak nesne segmentasyonunu kolaylaştırır.
shifted = cv2.pyrMeanShiftFiltering(image, 21, 51)

gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)

thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv2.imshow("thresh", thresh)

# Her ikili (binary) pikselden en yakın sıfır piksele olan tam Öklid mesafesini hesapla
D = ndimage.distance_transform_edt(thresh)
# Mesafe haritasındaki zirve (tepe) noktalarını bul
localMax = peak_local_max(D, footprint=np.ones((21, 21)), labels=thresh)
# Yerel zirveler için bir maske oluştur
mask = np.zeros(D.shape, dtype=bool)
mask[tuple(localMax.T)] = True

# Yerel zirvelerde bağlı bileşen analizini gerçekleştir
markers, _ = ndimage.label(mask)
# Watershed algoritmasını uygula
labels = watershed(-D, markers, mask=thresh)
print(" {} benzersiz segment bulundu".format(len(np.unique(labels)) - 1))

# Watershed algoritması tarafından döndürülen benzersiz etiketler üzerinde döngü
for label in np.unique(labels):
    if label == 0:
        continue

    # Etiket bölgesi için bellek ayır ve maskede çiz
    labelMask = np.zeros(gray.shape, dtype="uint8")
    labelMask[labels == label] = 255

    # Maskedeki konturları tespit et ve en büyüğünü al
    cnts = cv2.findContours(labelMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    # Eğer kontur yoksa veya çok küçükse atla
    if len(cnts) == 0 or cv2.contourArea(cnts[0]) < 200:  # Minimum alanı artırarak daha iyi filtreleme yapabiliriz
        continue

    # En büyük konturu bul
    c = max(cnts, key=cv2.contourArea)

    ((x, y), r) = cv2.minEnclosingCircle(c)
    cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)
    cv2.putText(image, "#{}".format(label), (int(x) - 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

cv2.imshow("sonuc", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


 9 benzersiz segment bulundu
